In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from astropy.table import Table
import gravity_obs.p2_tools as p2t

In [3]:
from astroquery.simbad import Simbad

In [4]:
simbad = Simbad()
simbad.add_votable_fields("parallax", "propermotions", "otype", "velocity", 'H', "K", "G")

In [5]:
from astropy.table import hstack, Column

from gravity_obs import obs_utils, search_gaia_single

target_list = ["MHO 2", 'FP Tau']

result = Table()
for i, targe_name in enumerate(target_list):
    simbad_tb = simbad.query_object(targe_name)
    ra, dec = simbad_tb['ra'][0], simbad_tb['dec'][0]
    coord_colon = obs_utils.coord_degree_to_colon(ra, dec)
    simbad_tb.rename_column('ra', 'ra_2000')
    simbad_tb.rename_column('dec', 'dec_2000')
    simbad_filter = ['ra_2000', 'dec_2000', 'H', 'K']

    gaia_tb = search_gaia_single(ra = ra, dec = dec, radius=1)
    gaia_tb.rename_column('ra', 'ra_2016')
    gaia_tb.rename_column('dec', 'dec_2016')
    gaia_tb.rename_column('phot_rp_mean_mag', 'Grp')
    gaia_filter = ['Grp', 'parallax', 'pmra', 'pmdec', 'radial_velocity', 'ra_2016', 'dec_2016']

    if i == 0:
        result.add_column(Column(name="name", data=[], dtype=str))
        result.add_column(Column(name="ra_2000_hms", data=[], dtype=coord_colon[0].dtype, unit = 'J2000'))
        result.add_column(Column(name="dec_2000_dms", data=[], dtype=coord_colon[1].dtype, unit = 'J2000'))

        for name in simbad_filter:
            result.add_column(Column(name=name, data=[], dtype=simbad_tb[name].dtype, 
                                     unit = simbad_tb[name].unit))
        for name in gaia_filter:
            result.add_column(Column(name=name, data=[], dtype=gaia_tb[name].dtype, 
                                     unit = gaia_tb[name].unit))

    new_tb = hstack([simbad_tb[simbad_filter], gaia_tb[gaia_filter]])
    row = [targe_name, coord_colon[0], coord_colon[1]] + [new_tb[col][0] for col in new_tb.colnames]
    result.add_row(row)
    #print(name, result, gaia_tb)

/Users/yixiancao/anaconda3/envs/gvobs/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:2625: RuntimeWarning: invalid value encountered in do_format (vectorized)
  outputs = ufunc(*args, out=...)


INFO: Query finished. [astroquery.utils.tap.core]
INFO: Query finished. [astroquery.utils.tap.core]


In [6]:
result

name,ra_2000_hms,dec_2000_dms,ra_2000,dec_2000,H,K,Grp,parallax,pmra,pmdec,radial_velocity,ra_2016,dec_2016
,J2000,J2000,deg,deg,,,mag,mas,mas / yr,mas / yr,km / s,deg,deg
str6,str12,str14,float64,float64,float64,float64,float32,float64,float64,float64,float32,float64,float64
MHO 2,04:14:26.401,+28:05:59.636,63.6100039315,28.099898862589995,9.423999786376953,7.791999816894531,15.376802,7.632029543003263,9.888589479587575,-25.38592582877389,0.0,63.61005375340543,28.09978603626571
FP Tau,04:14:47.303,+26:46:26.414,63.697097118120006,26.774003997700003,9.175000190734863,8.873000144958496,11.568479,7.844095665315957,8.989825384713045,-22.657513948657588,25.926144,63.69714187092417,26.773903297413405


In [ ]:
from gravity_obs import ob_add_target
def single_on_for_target(grav, target, folder_name=None,
                         usename=False, verify=False):
    ao_mode = "LGS_VIS"
    tel_alpha = target['ra_2000_hms']
    tel_delta = target['dec_2000_dms']

    if usename:
        obname = f"{ao_mode}_{target['name']}".replace(' ', '_')
    else:
        obname = f"{ao_mode}_{tel_alpha.strip().split('.')[0].replace(':', '')}{tel_delta.strip().split('.')[0].replace(':', '')}_{target['Grp']:4.1f}_{target['K']:4.1f}".replace(' ', '_')


    ob, obV = grav.create_OB(obname, folder_name=folder_name, overwrite=True)
    obsDes = ob['obsDescription']
    obsDes['name'] = target['name']
    obsDes['userComments'] = 'Single on-axis observation test'

    ob_add_target(ob, target['name'], 
                    ra = target['ra_2000_hms'], 
                    dec = target['dec_2000_dms'],
                    pma = target['pmra'],
                    pmd = target['pmdec'])
    ob, obV = grav.save_OB(obname, folder_name=folder_name)


    dit, ndit = 3, 40
    grav.add_GRAVITY_single_onaxis_acq(name=obname, folder_name=folder_name,
                                        sc_name=target['name'],
                                        sc_kmag=target['K'],
                                        sc_hmag=target['H'],
                                        ngs_mag=float(target['Grp']),
                                        ao_mode=ao_mode,
                                        plx=target['parallax']/1e3,
                                        baseline=['UTs'])
    grav.add_GRAVITY_single_obs_exp(obname, folder_name=folder_name,
                                    dit=dit, ndit_obj=ndit, ndit_sky=ndit,)

    # optional verification
    if verify:
        obID = ob['obId']
        response, _ = grav.api.verifyOB(obID,True)
        if response['observable']:
            print('*** Congratulations. Your OB' , obID, ob['name'], 'is observable!')
        else:
            print('OB', obID, 'is >>not observable<<. See messages below.')
            print(' ', '\n  '.join(response['messages']))

In [16]:
tel_alpha = target['ra_2000_hms']
tel_delta = target['dec_2000_dms']
tel_alpha.split('.')[0], tel_delta.strip().split('.')[0]

('04:14:47', '+26:46:26')

In [8]:
grav = p2t.p2api_GRAVITY('116.29H7.001', 'GPLUSTeam', 'GPAOwFRINGES', no_warning=True)
root_folder = 'TTR-116.0046 October First Light'
grav.set_rootFolder(root_folder)

In [9]:
tb = result
tb.add_index(colnames='name')


In [ ]:
folder_name = 'onaxis_ob_test'

try:
    grav.get_folder(folder_name)
except Exception:
    grav.create_folder(folder_name, overwrite=True)

tb = result.copy()
for obj_name in tb['name']:
    print (obj_name)
    target = tb.loc[obj_name]
    single_on_for_target(grav, target, folder_name=folder_name)

MHO 2
FP Tau
